# How is Power and Energy Calculated for a Hydropower Project?

The power generated by pressurized fluid in a pipeline is proportional to density ($\rho$), gravity ($g$), volumetric flow ($Q$), the electro-mechanical efficiency ($\zeta$), and the hydraulic head ($H_{net}$).

$$P = \rho g H_{net} Q \zeta$$

The hydraulic head is the elevation difference between the water level at the intake and the centreline elevation of the turbine, minus the losses in the pipeline due to friction.  

For a run-of-river hydropower project, the flow ($Q$) is governed by what is available in the river, minus what must be left in the river to maintain the aquatic ecosystem.  There is no storage, so the run-of-river plant can't be turned up or down to maximize revenue when demand (and thus price) is greatest.  On the other hand, the sediment that helps support the aquatic ecosystem in the diversion reach is not impeded in the same way as in a storage reservoir project.

The electro-mechanical efficiency is the friction loss in the conversion of rotational energy of the turbine runner to the movement of electric charges.  The total electro-mechanical efficiency is roughly 80-85% for a Pelton-type turbine-generator assembly.